# Projeto: RabbitMQ

Vamos simular produção, envio e consumo de mensagens numa empresa onde o departamento de dados criou uma arquitetura para esse fim com RabbitMQ. Nessa estrutura, os departamentos são livres para decidir quais informações serão compartilhadas e o modo como isso acontecerá. É importante salientar que os departamentos também possuem funcionários dedicados à dados e as decisões sobre o modo de compartilhamento foram tomadas com base na análise de anos de pedidos recebidos por cada departamento.

## Produtores

Os departamentos, dados compartilhados e modo de compartilhamento são:

*  P1: `IT`
      1. Todos os logs são publicados no modo *fanout*.
      2. Todos erros de alta prioridade em aberto são publicados no modo *topic*, rota *department.category.priority.status*.

* P2: `SALES`
    1. Todos orçamentos  são publicados no modo *direct*, rota *orcamentos*.
    2. Todas ordens de compra  são publicados no modo *direct*, rota *ordens*.

* P3: `MKT`
    1. Eventos importantes (prioridade alta) são publicados no modo *direct*, rota *eventos_importantes*.
    2. Todas as promoções são publicadas no modo *topic*, rota *department.category.priority*.

* P4: `RH`
    1. Todos os eventos são publicados no modo *fanout*.
    2. Eventos importantes (prioridade alta) são publicados no modo *direct*, rota *eventos_importantes*.
    3. Todas as movimentações são publicadas no modo *topic*, rota *department.category.priority*.

## Arquitetura

A arquitetura consiste de um servidor RabbitMq hospedado em `localhost`, que suporta três *exchanges* e XXX filas.

### Exchanges

* E1: `fanout`
  1. nome: exchange_fanout
  2. tipo: fanout
* E2: `direct`
  1. nome: exchange_direct
  2. tipo: direct
* E3: `topic`
  1. nome: exchange_topic
  2. tipo: topic


### Filas

As filas que fazem parte da arquitetura foram criadas para garantir a entrega das mensagens a certos destinatários.  Para isso, elas devem ser à prova de falhas.

* Q1: `informacoes_criticas`  
  1. Criada para **garantir** que as mensagens de alta prioridade e abertas oriundas de TI **sejam entregues** ao time de gerentes de projetos, permitindo que eles tenham tempo suficiente para reagir às informações. 
  2. Um time processa todas as informações.
* Q2: `participacao_obrigatoria`  
  1. Criada para garantir que que os lideres de cada departamento sejam notificados dos eventos que devem participar. 
  2. Cada líder recebe sua mensagem. 
* Q3: `compras`
  1. Criada para operações do time de Sales de forma a organizar a execução das  tarefas, essa fila garante que nenhum pedido seja perdido.
  2. Um time recebe todas as informações.
* Q4: `logs`
  1. Criada pelo próprio time de dados para registar o fluxo de mensagens da arquitetura e facilitar a resolução de possíveis conflitos sobre mensagens não enviadas/entregues.
  2. Um time recebe todas as mensagens.

# Consumidores

Com essa arquitetura moderna e flexível diversos consumidores podem consumir informações dos mesmo produtores, sem  depender de intermediários. Alguns consumidores e suas necessidades são:

* `C1`: time de projetos
  1. Esse consumidor lê da fila `Q1`, ligada à `B1` e demora de 5 a 10 segundos para processar cada mensagem.
* `C2`: lideres
  1. Esse(s) consumidor(es) lê(em) da(s) fila(s) `Q2`, ligada(s) à(s) `B2` e demora(m) de 2 a 4 segundos para processar cada mensagem.
  2. Cada líder deve receber suas mensagens.
* `C3`: time de compras
  1. Esse consumidor lê da fila `Q3`, ligada à `B3` e demora(m) de 10 a 20 segundos para processar cada mensagem.
  2. O mesmo time lê todas as mensagens.
* `C4`: time de dados
  1. Esse consumidor lê da fila `Q4`, ligada à `B4` e demora(m) de 1 a 20 segundos para processar cada mensagem.
  2. O mesmo time lê todas as mensagens.

Esse *setup* inicial foi muito bem sucedido e outras equipes mostraram interesse em receber informações dessa maneira. Para evitar a sobrecarga do sistema e de tarefas para o time e aumentar a flexibilidade e tempo de desenvolvimento, ficou decidido que esses consumidores receberiam as mensagens em filas temporárias.

* `C5`: time de eventos
  1. Recebe todas as mensagens do *exchange: fanout* e publica um email automatizado sobre essas informações para o restante da empresa. 
* `C6`: time de experiência
  1. Recebe e analisa todas as mensagens de prioridade alta.
* `C7`: time de plataformas
  1. Recebe e analisa todas as mensagens de TI, as movimentações do RH e todas as mensagens de Sales.

# Preparação

Faça um diagrama esquemático da arquitetura apresentada acima.

[diagrama_exercicio]()

# Exemplo guiado

1. Crie o produtor `P1`, o exchange `E3`, a fila `Q1` e o binding `B1`, nessa ordem.
2. Execute o produtor.
3. Crie o consumer `C1`.

**Pergunta**: A partir de qual mensagem ele começou a consumir?

4. Derrube `C1` e inicie novamente após alguns instantes.

**Pergunta**: Ele está consumindo da última mensagem enviada ou a partir de onde havia parado?

# Tarefas

1. Crie os três exchanges. Use os arquivos disponibilizados como guia.
2. Crie os quatro produtores. Use os arquivos disponibilizados como guia e note que o tempo entre uma mensagem e outra é aleatório.

**Pergunta**: Com a primeira parte da arquitetura finalizada, somos capazes de gerar e distribuir mensagens. Dispare algumas delas a partir de cada produtor. Recebeu algum erro? Para onde foram as enviadas as mensagens geradas dessa forma?


3. Crie as filas que são parte da arquitetura, mas não faça as ligações por enquanto.

**Pergunta**: Uma única fila com cada label (`Q1`, `Q2`, `Q3` e `Q4`) são suficientes para atender a demanda dos respectivos consumidores?

**Pergunta**: Crie e envie algumas mensagens. Elas chegaram às filas? Verifique via CLI ou UI.

4. Crie as ligações apropriadas (`B1` a `B4`) para cada uma das filas. 

**Pergunta**: Crie e envie algumas mensagens. Elas chegaram às filas? Verifique via CLI ou UI.

5. Crie o(s) consumidor(es) `C2` (lideres). 

6. Crie um consumidor `C3` (compras). 

7. Crie um consumidor `C4` (dados). 

8. Ajuste os quatro produtores para mandar 10_000 mensagens e coloque-os para trabalhar. Certifique-se que todas as mensagens chegam aos consumidores corretos.

9. Crie os consumidores `C5` a `C7` com suas respectivas filas e ligações temporárias.

10. Ligue todos os consumidores e certifique-se que cada um deles recebe as mensganes que deveria.

# Extras

***Em desenvolvimento***